In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [ ]:
# 2. Create a temporary view of the DataFrame.


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


In [ ]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached


In [ ]:

import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round


In [ ]:

spark = SparkSession.builder.appName("HomeSales").getOrCreate()


In [ ]:

df = spark.read.csv("s3a://amazon-reviews-ml-pipeline/home_sales_revised.csv", header=True, inferSchema=True)
df.show(5)


In [ ]:

df.createOrReplaceTempView("home_sales")


In [ ]:

query1 = spark.sql("""
    SELECT year(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year_sold
    ORDER BY year_sold
""")
query1.show()


In [ ]:

query2 = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")
query2.show()


In [ ]:

query3 = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")
query3.show()


In [ ]:

start = time.time()
query4 = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view
""")
query4.show()
print("Uncached Runtime:", time.time() - start)


In [ ]:

spark.catalog.cacheTable("home_sales")


In [ ]:

print("Is 'home_sales' cached?:", spark.catalog.isCached("home_sales"))


In [ ]:

start = time.time()
query4_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view
""")
query4_cached.show()
print("Cached Runtime:", time.time() - start)


In [ ]:

df.write.partitionBy("date_built").parquet("/tmp/home_sales_partitioned", mode="overwrite")


In [ ]:

parquet_df = spark.read.parquet("/tmp/home_sales_partitioned")
parquet_df.createOrReplaceTempView("parquet_home_sales")


In [ ]:

start = time.time()
query4_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM parquet_home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view
""")
query4_parquet.show()
print("Parquet Query Runtime:", time.time() - start)


In [ ]:

spark.catalog.uncacheTable("home_sales")
print("Is 'home_sales' cached after uncache?:", spark.catalog.isCached("home_sales"))
